In [1]:
from sklearn.datasets import load_digits
from sklearn.cross_validation import train_test_split
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer


#load data
digits = load_digits()
X = digits.data
y = digits.target
y = LabelBinarizer().fit_transform(y)
#                 ^^常忘記     

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.3)

#define placeholder for inputs on network
keep_prob = tf.placeholder(tf.float32)  #<----------------------------新增
Xs = tf.placeholder(tf.float32,[None,64])
ys = tf.placeholder(tf.float32,[None,10])


# define add_layer
def add_layer(inputs, in_size, out_size, layer_name, activation_function=None):
    Weights = tf.Variable(tf.random_normal([in_size,out_size]))
    baises = tf.Variable(tf.zeros([1,out_size]) + 0.1)
    Wx_plus_b = tf.matmul(inputs,Weights) + baises
    Wx_plus_b = tf.nn.dropout(Wx_plus_b,keep_prob)  #<----------------------------新增，重點！
    
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b,)
    #recrod outputs
    tf.histogram_summary(layer_name + '/outputs',outputs)
    
    return outputs

#add more layers and return the output of add_layer
L1 = add_layer(Xs, 64, 50, 'L1', activation_function=tf.nn.tanh)   #<----新function,測試overfitting用
prediction = add_layer(L1, 50, 10, 'prediction', activation_function=tf.nn.softmax)

#the loss
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),reduction_indices=[1]))
#record summary
tf.scalar_summary('loss',cross_entropy)

#Optimizer
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

#active session
sess = tf.Session()
init = tf.initialize_all_variables()
        #^^^^^^^^^^^一直拼錯
sess.run(init)
#merge summary
merged = tf.merge_all_summaries()

#summary writer
train_writer = tf.train.SummaryWriter('logs/train',sess.graph)
                #^^^^^常忘記^^^^^^^^^要大寫
test_writer = tf.train.SummaryWriter('logs/test',sess.graph)

    
#training cross_entropy
for i in range(500):
    sess.run(train_step, feed_dict={Xs: X_train, ys: y_train, keep_prob:0.6}) #<-----------是冒號不是等於-----保留60%丟去40%
    if i % 50 ==0:
        #use data_train and data_test recrod train_result
        train_result = sess.run(merged, feed_dict={Xs: X_train, ys: y_train, keep_prob: 1})  #<----------------要完整紀錄
        test_result = sess.run(merged, feed_dict={Xs: X_test, ys: y_test, keep_prob: 1})  #<------------------要完整紀錄
        #writting train_result in writer
        train_writer.add_summary(train_result, i)
        test_writer.add_summary(test_result, i)
        

//anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Instructions for updating:
Please switch to tf.summary.histogram. Note that tf.summary.histogram uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on their scope.
Instructions for updating:
Please switch to tf.summary.histogram. Note that tf.summary.histogram uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on their scope.
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Please switch to tf.summary.merge_all.
Instructions for updating:
Please switch to tf.summary.merge.
Instruc